# 提示词要素
提示词（Prompt）是向大语言模型传递任务指令的文本，其设计质量直接影响模型输出的效果，提示词，尤其是指令的构建需要注意一些核心要素，掌握了这些核心要素，大模型就能够不经过微调训练等得到符合预期的回复。

分别有下面的要素
1. 明确任务类型
2. 明确任务目标
3. 提供上下文信息
4. 结构化提示词
5. 复杂任务拆解

In [3]:
### 加载模型：Qwen2.5-0.5B-Instruct模型
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = '/your/path/of/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,device_map='auto', torch_dtype='auto')

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.19it/s]


In [4]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

## 1. 明确任务类型

要非常具体地说明你希望模型执行的指令和任务。提示越具描述性和详细，结果越好。特别是当你对生成的结果或风格有要求时，这一点尤为重要。

任务类型有文本翻译、指令问答、多轮对话、文本摘要、文本分类、文本生成等。

我们举一个简单的文本翻译的例子。

In [9]:
prompt_1="""
请将下面的话翻译成英文：

输入：
大模型是参数量很大，功能很强，但是推理速度慢，成本高。

输出：

"""

In [10]:
print(inference(model,tokenizer,prompt_1))

{'response': 'Large models have a very large number of parameters, powerful functionality, but slow inference speed and high costs.', 'generated_tokens_count': 22}


当然，我们也可以举一个文本分类的例子，我们写一个完整的提示词。

In [7]:
prompt_2="""

请你担任文本情绪分类员，帮我将以下文本按情绪类型分类。分类规则如下：

正面情绪：文本表达积极、愉悦、赞赏、满意等正向感受（如 “这部电影太精彩了，我非常喜欢！”）；
负面情绪：文本表达消极、不满、厌恶、愤怒等负向感受（如 “这个服务太差劲了，再也不会来了”）；
中性情绪：文本不包含明显的情绪倾向，仅陈述事实或客观信息（如 “会议将在下午 3 点开始”）。

请对以下文本进行分类，并只输出分类结果（格式：文本内容 + 【分类标签】）：
文本 1：今天天气真好，适合出门野餐
文本 2：这台机器坏了，修了好几次都没好
文本 3：本市明天最高气温 25 摄氏度
"""

In [8]:
print(inference(model,tokenizer,prompt_2))

{'response': '文本 1：今天天气真好，适合出门野餐【正面情绪】  \n文本 2：这台机器坏了，修了好几次都没好【负面情绪】  \n文本 3：本市明天最高气温 25 摄氏度【中性情绪】', 'generated_tokens_count': 59}


## 2. 明确任务目标

在明确了任务类型，比如是问答类型的话，你可能需要对任务目标加以约束，比如让大模型做什么、不做什么。下面我们举个例子。

大模型由于知识广泛，回答问题的时候发散性思维比较强，因此有时候的回答会带有他的分析，比如我们想要计算一个简单的数学问题，并且只想要得到结果的数字，但是简单的提示词可能会有大模型自己的分析，如果对输出格式或者结果有要求，一定要明确表明。

In [11]:
prompt_3="请你计算出3*13的结果"
prompt_4="请你计算出3*13的结果，仅输出结果，不要输出任何其他内容。"

In [12]:
print(inference(model,tokenizer,prompt_3))
print(inference(model,tokenizer,prompt_4))

{'response': '3乘以13的结果是39。', 'generated_tokens_count': 11}
{'response': '39', 'generated_tokens_count': 3}


## 3. 提供上下文信息

- 提供充足的文本信息或者背景知识，这点在文本摘要、信息提取任务中尤为重要
- 少样本提示的话需要一些例子作为前文信息

In [17]:
prompt_5="""
请你根据上下文内容，回答输入提出的问题，回答到输出里。
上下文：2025年7月8日，国家航天局宣布，嫦娥七号探测器计划于2026年发射，将开展月球南极环境与资源勘查。据悉，嫦娥七号将搭载4台国际合作载荷，包括与欧洲空间局合作的月表负离子分析仪。
输入：嫦娥七号的发射时间
输出：
"""

prompt_6="""
Q: 苹果公司成立于哪一年？  
A: 1976  

Q: 爱因斯坦获得诺贝尔物理学奖是在哪一年？  
A:
"""

In [18]:
print(inference(model,tokenizer,prompt_5))
print(inference(model,tokenizer,prompt_6))

{'response': '嫦娥七号计划于2026年发射。', 'generated_tokens_count': 14}
{'response': '爱因斯坦获得诺贝尔物理学奖的年份是1921年。', 'generated_tokens_count': 18}


## 4. 结构化提示词

我们在使用提示词的时候往往会直接把文本、问题等一股脑全部输入到大模型的输入中，但是实际上我们可以通过严格控制提示词的格式来提升模型输出的质量，因为结构化的提示词汇更具可控性，尤其是在需要精确格式或复杂任务的场景中。

我们通常使用markdown格式来结构化。Markdown 是一种轻量级标记语言，常用于格式化文本（如表格、列表、代码块）。结构化提示词可以使用 Markdown 语法来表达，但核心是 “结构化” 的思想，而非特定语言。

In [19]:
prompt_7="""
# 任务：提取电影信息
请从以下文本中提取关键信息，按要求填写：

## 待分析文本
"电影《流浪地球3》将于2027年春节上映，导演是郭帆，主演包括吴京和刘德华，票房预期50亿。"

## 提取要求
- 类型：______
- 上映时间：______
- 导演：________
- 主演：______
- 票房预期：______

## 输出格式
直接填写上述空白，保持列表结构。
"""

In [21]:
print(inference(model,tokenizer,prompt_7))

{'response': '- 类型：电影\n- 上映时间：2027年春节\n- 导演：郭帆\n- 主演：吴京，刘德华\n- 票房预期：50亿', 'generated_tokens_count': 48}


## 5. 复杂任务拆解

大模型和人类一样，哪怕在现实生活中，如果Leader给你分配了一个比较复杂、内容较多的任务，经验丰富的你大概率会先列出任务细节，也就是大纲，然后对着大纲的每一步进行细化，比如实验步骤、完成时间、分配员工等，细化完后就开始每一步的完成细节了。


In [22]:
prompt_8="""
请你担任任务规划师，帮我把 “3 个月内自学 Python 并完成一个数据分析项目” 这个任务拆解成具体、可操作的步骤。
请按照 “阶段划分→核心目标→具体步骤→成果物→难点建议” 的结构输出，确保步骤之间有逻辑衔接，且难度循序渐进。
"""

In [24]:
print(inference(model,tokenizer,prompt_8,max_new_tokens=2048))

{'response': '当然可以。根据您的需求，“3 个月内自学 Python 并完成一个数据分析项目”的任务可以分为以下阶段：\n\n### 阶段划分\n1. **基础知识阶段**（第1-4周）\n2. **编程技能提升阶段**（第5-8周）\n3. **项目开发与优化阶段**（第9-12周）\n\n### 第一阶段：基础知识阶段（第1-4周）\n\n#### 核心目标\n- 掌握 Python 基础语法和数据类型。\n- 熟悉常用的 Python 数据库操作库。\n- 学习基本的数据分析方法和技巧。\n\n#### 具体步骤\n1. **基础语法学习**：\n   - 安装并配置 Python 开发环境。\n   - 学习 Python 基本语法、数据类型（如整数、浮点数、字符串等）、变量、条件语句、循环语句等。\n   - 使用 Python 编写简单的程序，熟悉 IDE（如 PyCharm、Jupyter Notebook）的使用。\n\n2. **安装和配置工具**：\n   - 安装常用的数据处理库（如 NumPy, Pandas, Matplotlib 等）。\n   - 设置版本控制系统（如 Git），了解基本操作。\n\n3. **数据库操作**：\n   - 学习如何使用 SQLite 或 MySQL 读取、插入、修改、删除数据。\n   - 使用 Python 连接数据库并执行 SQL 查询。\n\n4. **数据清洗和预处理**：\n   - 学习如何使用 Pandas 处理数据，包括加载数据、数据清理、缺失值处理等。\n\n5. **数据分析与可视化**：\n   - 学习基本的统计方法和可视化工具（如 Matplotlib、Seaborn）。\n   - 分析示例数据集，进行简单的统计分析和数据可视化。\n\n#### 成果物\n- 基础知识练习题完成情况报告。\n- 数据库连接和基本数据操作的小项目。\n- 基本的数据清洗和简单数据分析案例。\n\n#### 难点建议\n- 对于初学者来说，理解 Python 语法和概念可能较为困难。建议多做练习，并通过查阅官方文档和教程加深理解。\n- 数据库操作可能会遇到一些技术性问题，需要耐心学习和实践。\n\n### 第二阶段：编程技能提升阶段（第5-8周）\n\n#### 核心目标\n- 